<a href="https://colab.research.google.com/github/Amirhatamian/NLP/blob/main/NLP_ASSIGNMENTinitialapproch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# Install necessary libraries
!pip install wikipedia nltk scikit-learn PyPDF2 Wikipedia-API


In [38]:
#Import Libraries and Download NLTK Data

import re
import string
import numpy as np
import wikipedia
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import PyPDF2

# Download NLTK data
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [39]:
# Initialize the Wikipedia API for the English language
wiki_wiki = wikipediaapi.Wikipedia('english')

# Function to retrieve text from a Wikipedia page
def get_wikipedia_text(page_title):
    # Retrieve the page object for the given title
    page = wiki_wiki.page(page_title)

    # Check if the page exists
    if not page.exists():
        return None

    # Return the text content of the page
    return page.text


In [40]:
def extract_keywords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word.lower() for word in words if word.isalnum() and word.lower() not in stop_words]
    return filtered_words


In [41]:
def extract_nouns(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    tagged_words = pos_tag(words)
    nouns = [word.lower() for word, pos in tagged_words if pos.startswith('N') and word.lower() not in stop_words and word.isalnum()]
    return nouns


In [42]:
def get_wikipedia_text(topic):
    try:
        # Get the summary of the Wikipedia page for the topic
        summary = wikipedia.summary(topic)
        return summary
    except wikipedia.exceptions.DisambiguationError as e:
        # Handle disambiguation errors by choosing the first option
        return wikipedia.summary(e.options[0])
    except wikipedia.exceptions.PageError:
        # Handle page errors (e.g., page not found)
        return None

def extract_nouns(text):
    # Tokenize the text into words
    words = word_tokenize(text)
    # Tag words with part of speech
    pos_tags = nltk.pos_tag(words)
    # Extract nouns (NN, NNS, NNP, NNPS are the tags for different types of nouns)
    nouns = [word for word, pos in pos_tags if pos in ['NN', 'NNS', 'NNP', 'NNPS']]
    return nouns

def extract_top_nouns(topics, num_top_nouns=10):
    all_nouns = []

    for topic in topics:
        text = get_wikipedia_text(topic)
        if text:
            all_nouns.extend(extract_nouns(text))

    # Remove common stopwords from the list of nouns
    stop_words = set(stopwords.words('english'))
    filtered_nouns = [noun for noun in all_nouns if noun.lower() not in stop_words]

    # Create a frequency distribution of the nouns
    nouns_freq_dist = FreqDist(filtered_nouns)
    # Get the most common nouns
    top_nouns = [word for word, _ in nouns_freq_dist.most_common(num_top_nouns)]

    return top_nouns

In [43]:
#
geographic_topics = ['New York', 'Mount Everest', 'Sahara Desert', 'Amazon River', 'Paris']
non_geographic_topics = ['Quantum Mechanics', 'Artificial Intelligence', 'Shakespeare', 'Modern Art', 'Jazz Music']


In [44]:
top_geographic_keywords = extract_top_nouns(geographic_topics, num_top_nouns=10)
top_non_geographic_keywords = extract_top_nouns(non_geographic_topics, num_top_nouns=10)

print("Top geographic keywords:", top_geographic_keywords)
print("Top non-geographic keywords:", top_non_geographic_keywords)

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Top geographic keywords: ['New', 'York', 'Paris', 'city', 'world', 'state', 'River', 'Amazon', 'United', 'area']
Top non-geographic keywords: ['AI', 'jazz', 'art', 'Shakespeare', 'systems', 'intelligence', 'works', 'physics', 'quantum', 'mechanics']


In [45]:
# Combine all topics
all_topics = geographic_topics + non_geographic_topics
all_docs = []
all_labels = []

# Fetch text and extract nouns for each topic
for topic in all_topics:
    text = get_wikipedia_text(topic)
    if text:
        nouns = extract_nouns(text)
        all_docs.append(" ".join(nouns))
        # Label as 1 for geographic and 0 for non-geographic
        all_labels.append(1 if topic in geographic_topics else 0)

# Combine top keywords from both categories
all_top_keywords = top_geographic_keywords + top_non_geographic_keywords

# Vectorize the documents using the top keywords
vectorizer = CountVectorizer(vocabulary=all_top_keywords)
X = vectorizer.transform(all_docs)
y = all_labels

print("Feature matrix shape:", X.shape)
print("Labels:", y)

Feature matrix shape: (9, 20)
Labels: [1, 1, 1, 1, 0, 0, 0, 0, 0]


In [46]:
#Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [47]:
# Majority class naive classifier
class NaiveClassifier:
    def __init__(self):
        self.majority_class = None

    def fit(self, X, y):
        unique_classes, counts = np.unique(y, return_counts=True)
        self.majority_class = unique_classes[np.argmax(counts)]

    def predict(self, X):
        return np.full(X.shape[0], self.majority_class)

# Instantiate and train the naive classifier
naive_classifier = NaiveClassifier()
naive_classifier.fit(X_train, y_train)

# Predict on the test set
naive_predictions = naive_classifier.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, naive_predictions)
report = classification_report(y_test, naive_predictions, target_names=['non-geographic', 'geographic'])

print(f'Accuracy: {accuracy * 100:.2f}%')
print('Classification Report:')
print(report)


Accuracy: 50.00%
Classification Report:
                precision    recall  f1-score   support

non-geographic       0.50      1.00      0.67         1
    geographic       0.00      0.00      0.00         1

      accuracy                           0.50         2
     macro avg       0.25      0.50      0.33         2
  weighted avg       0.25      0.50      0.33         2



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [48]:
# Specify the path to the PDF file on your desktop
pdf_file_path = "path_to_your_pdf_file.pdf"

# Read the content of the PDF file
with open(pdf_file_path, 'rb') as file:
    pdf_reader = PyPDF2.PdfReader(file)
    pdf_text = ''
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        pdf_text += page.extract_text()

# Vectorize the PDF document
document_vectorized = vectorize_document(pdf_text, vectorizer)

# Predict the class of the PDF document
naive_prediction = naive_classifier.predict(document_vectorized)

if naive_prediction[0] == 1:
    print('The document is classified as geographic.')
else:
    print('The document is classified as non-geographic.')

FileNotFoundError: [Errno 2] No such file or directory: 'path_to_your_pdf_file.pdf'